# Setup

In [1]:
REDIS_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

REDIS_TOTAL_NODES = 6
REDIS_NODE_IPS = ["10.15.20.2"] * REDIS_TOTAL_NODES
REDIS_NODE_NAMES = [f"redis-node-{i+1}" for i in range(REDIS_TOTAL_NODES)]
REDIS_NODE_HOSTNAMES = [
    f"{REDIS_NODE_NAMES[i]}.mavasbel.vpn.itam.mx" for i in range(REDIS_TOTAL_NODES)
]
REDIS_NODE_PORTS = [f"{6380 + i + 1}" for i in range(REDIS_TOTAL_NODES)]
REDIS_NODE_BUS_PORTS = [f"{16380 + i + 1}" for i in range(REDIS_TOTAL_NODES)]

REDIS_WORKDIR = "/data"

REDIS_ADMIN_PASSWORD = "redis"
REDIS_DEFAULT_PASSWORD = "redis"
REDIS_INIT_USER = "redis"
REDIS_INIT_PASSWORD = "redis"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)

# Stop redis-cluster.docker-compose.yml

In [3]:
!docker compose -f redis-cluster.docker-compose.yml down -v

 Container redis-node-3  Stopping
 Container redis-node-1  Stopping
 Container redis-node-4  Stopping
 Container redis-node-6  Stopping
 Container redis-node-2  Stopping
 Container redis-node-5  Stopping
 Container redis-node-4  Stopped
 Container redis-node-4  Removing
 Container redis-node-4  Removed
 Container redis-node-5  Stopped
 Container redis-node-5  Removing
 Container redis-node-5  Removed
 Container redis-node-6  Stopped
 Container redis-node-6  Removing
 Container redis-node-6  Removed
 Container redis-node-3  Stopped
 Container redis-node-3  Removing
 Container redis-node-3  Removed
 Container redis-node-2  Stopped
 Container redis-node-2  Removing
 Container redis-node-2  Removed
 Container redis-node-1  Stopped
 Container redis-node-1  Removing
 Container redis-node-1  Removed
 Network redis-cluster_redis-cluster  Removing
 Network redis-cluster_redis-cluster  Removed


In [4]:
import shutil

if REDIS_START_FROM_SCRATCH:
    shutil.rmtree(DOCKER_MOUNTDIR, ignore_errors=True)
    Path(DOCKER_MOUNTDIR).mkdir(parents=True, exist_ok=True)

# Start opensearch-cluster.docker-compose.yml

In [5]:
import os
import yaml
from IPython.display import Markdown, display


REDIS_MEMORY_LIMIT = "512mb"
REDIS_CPU_LIMIT = "0.5"

redis_compose_dict = {
    "name": "redis-cluster",
    "services": {},
    "networks": {
        "redis-cluster": {
            "driver": "bridge",
            # "ipam": {
            #     "config": [{"subnet": "173.20.0.0/16"}]
            # }
        }
    },
}

for i in range(REDIS_TOTAL_NODES):
    redis_compose_dict["services"][REDIS_NODE_NAMES[i]] = {
        "image": "redis:8.4.0-bookworm",
        "container_name": REDIS_NODE_NAMES[i],
        "command": [
            "redis-server",
            *["--cluster-enabled", "yes"],
            *["--requirepass", f"{REDIS_DEFAULT_PASSWORD}"],
            *["--masterauth", f"{REDIS_ADMIN_PASSWORD}"],
            *[
                "--user",
                f"{REDIS_INIT_USER}",
                "on",
                f">{REDIS_INIT_PASSWORD}",
                "~*",
                "+@all",
            ],
            *["--cluster-node-timeout", "15000"],
            *["--cluster-config-file", "nodes.conf"],
            *["--cluster-node-timeout", "5000"],
            *["--appendonly", "yes"],
            *["--cluster-replica-validity-factor", "0"],
            *["--port", REDIS_NODE_PORTS[i]],
            *["--bind", "0.0.0.0"],
            *["--protected-mode", "no"],
            *["--cluster-announce-ip", REDIS_NODE_IPS[i]],
            *["--cluster-announce-port", REDIS_NODE_PORTS[i]],
            *["--cluster-announce-bus-port", REDIS_NODE_BUS_PORTS[i]],
        ],
        "volumes": [
            f"{os.path.join(DOCKER_MOUNTDIR, REDIS_NODE_NAMES[i])}:{REDIS_WORKDIR}"
        ],
        "networks": ["redis-cluster"],
        # f"hostname": f"{REDIS_NODE_HOSTNAMES[i]}",
        "ports": [
            f"{REDIS_NODE_PORTS[i]}:{REDIS_NODE_PORTS[i]}",
            f"{REDIS_NODE_BUS_PORTS[i]}:{REDIS_NODE_BUS_PORTS[i]}",
        ],
        "dns": DOCKER_DNS,
        "deploy": {
            "resources": {
                "limits": {"cpus": REDIS_CPU_LIMIT, "memory": REDIS_MEMORY_LIMIT}
            }
        },
        "depends_on": {
            REDIS_NODE_NAMES[j]: {"condition": "service_started"} for j in range(0, i)
        },
        "healthcheck": {
            "test": ["CMD", "redis-cli", "-p", f"{REDIS_NODE_PORTS[i]}", "ping"],
            "interval": "10s",
            "timeout": "10s",
            "retries": 10,
            "start_period": "20s",
        },
    }

redis_compose_yaml_path = os.path.join(
    LOCALHOST_WORKDIR, "redis-cluster.docker-compose.yml"
)
redis_compose_yaml_contents = yaml.dump(
    redis_compose_dict, default_flow_style=False, sort_keys=False, indent=4
)

with open(redis_compose_yaml_path, "w") as f:
    f.write(redis_compose_yaml_contents)

print(f"Successfully created: '{os.path.abspath(redis_compose_yaml_path)}'")
display(Markdown(f"```yaml\n{redis_compose_yaml_contents}\n```"))

Successfully created: 'c:\Users\Marco\Documents\ITAM\2026 - Bases de Datos No Relacionales\code\redis\redis-cluster.docker-compose.yml'


```yaml
name: redis-cluster
services:
    redis-node-1:
        image: redis:8.4.0-bookworm
        container_name: redis-node-1
        command:
        - redis-server
        - --cluster-enabled
        - 'yes'
        - --requirepass
        - redis
        - --masterauth
        - redis
        - --user
        - redis
        - 'on'
        - '>redis'
        - ~*
        - +@all
        - --cluster-node-timeout
        - '15000'
        - --cluster-config-file
        - nodes.conf
        - --cluster-node-timeout
        - '5000'
        - --appendonly
        - 'yes'
        - --cluster-replica-validity-factor
        - '0'
        - --port
        - '6381'
        - --bind
        - 0.0.0.0
        - --protected-mode
        - 'no'
        - --cluster-announce-ip
        - 10.15.20.2
        - --cluster-announce-port
        - '6381'
        - --cluster-announce-bus-port
        - '16381'
        volumes:
        - .\mount\redis-node-1:/data
        networks:
        - redis-cluster
        ports:
        - 6381:6381
        - 16381:16381
        dns: &id001
        - 10.15.20.1
        deploy:
            resources:
                limits:
                    cpus: '0.5'
                    memory: 512mb
        depends_on: {}
        healthcheck:
            test:
            - CMD
            - redis-cli
            - -p
            - '6381'
            - ping
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 20s
    redis-node-2:
        image: redis:8.4.0-bookworm
        container_name: redis-node-2
        command:
        - redis-server
        - --cluster-enabled
        - 'yes'
        - --requirepass
        - redis
        - --masterauth
        - redis
        - --user
        - redis
        - 'on'
        - '>redis'
        - ~*
        - +@all
        - --cluster-node-timeout
        - '15000'
        - --cluster-config-file
        - nodes.conf
        - --cluster-node-timeout
        - '5000'
        - --appendonly
        - 'yes'
        - --cluster-replica-validity-factor
        - '0'
        - --port
        - '6382'
        - --bind
        - 0.0.0.0
        - --protected-mode
        - 'no'
        - --cluster-announce-ip
        - 10.15.20.2
        - --cluster-announce-port
        - '6382'
        - --cluster-announce-bus-port
        - '16382'
        volumes:
        - .\mount\redis-node-2:/data
        networks:
        - redis-cluster
        ports:
        - 6382:6382
        - 16382:16382
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '0.5'
                    memory: 512mb
        depends_on:
            redis-node-1:
                condition: service_started
        healthcheck:
            test:
            - CMD
            - redis-cli
            - -p
            - '6382'
            - ping
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 20s
    redis-node-3:
        image: redis:8.4.0-bookworm
        container_name: redis-node-3
        command:
        - redis-server
        - --cluster-enabled
        - 'yes'
        - --requirepass
        - redis
        - --masterauth
        - redis
        - --user
        - redis
        - 'on'
        - '>redis'
        - ~*
        - +@all
        - --cluster-node-timeout
        - '15000'
        - --cluster-config-file
        - nodes.conf
        - --cluster-node-timeout
        - '5000'
        - --appendonly
        - 'yes'
        - --cluster-replica-validity-factor
        - '0'
        - --port
        - '6383'
        - --bind
        - 0.0.0.0
        - --protected-mode
        - 'no'
        - --cluster-announce-ip
        - 10.15.20.2
        - --cluster-announce-port
        - '6383'
        - --cluster-announce-bus-port
        - '16383'
        volumes:
        - .\mount\redis-node-3:/data
        networks:
        - redis-cluster
        ports:
        - 6383:6383
        - 16383:16383
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '0.5'
                    memory: 512mb
        depends_on:
            redis-node-1:
                condition: service_started
            redis-node-2:
                condition: service_started
        healthcheck:
            test:
            - CMD
            - redis-cli
            - -p
            - '6383'
            - ping
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 20s
    redis-node-4:
        image: redis:8.4.0-bookworm
        container_name: redis-node-4
        command:
        - redis-server
        - --cluster-enabled
        - 'yes'
        - --requirepass
        - redis
        - --masterauth
        - redis
        - --user
        - redis
        - 'on'
        - '>redis'
        - ~*
        - +@all
        - --cluster-node-timeout
        - '15000'
        - --cluster-config-file
        - nodes.conf
        - --cluster-node-timeout
        - '5000'
        - --appendonly
        - 'yes'
        - --cluster-replica-validity-factor
        - '0'
        - --port
        - '6384'
        - --bind
        - 0.0.0.0
        - --protected-mode
        - 'no'
        - --cluster-announce-ip
        - 10.15.20.2
        - --cluster-announce-port
        - '6384'
        - --cluster-announce-bus-port
        - '16384'
        volumes:
        - .\mount\redis-node-4:/data
        networks:
        - redis-cluster
        ports:
        - 6384:6384
        - 16384:16384
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '0.5'
                    memory: 512mb
        depends_on:
            redis-node-1:
                condition: service_started
            redis-node-2:
                condition: service_started
            redis-node-3:
                condition: service_started
        healthcheck:
            test:
            - CMD
            - redis-cli
            - -p
            - '6384'
            - ping
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 20s
    redis-node-5:
        image: redis:8.4.0-bookworm
        container_name: redis-node-5
        command:
        - redis-server
        - --cluster-enabled
        - 'yes'
        - --requirepass
        - redis
        - --masterauth
        - redis
        - --user
        - redis
        - 'on'
        - '>redis'
        - ~*
        - +@all
        - --cluster-node-timeout
        - '15000'
        - --cluster-config-file
        - nodes.conf
        - --cluster-node-timeout
        - '5000'
        - --appendonly
        - 'yes'
        - --cluster-replica-validity-factor
        - '0'
        - --port
        - '6385'
        - --bind
        - 0.0.0.0
        - --protected-mode
        - 'no'
        - --cluster-announce-ip
        - 10.15.20.2
        - --cluster-announce-port
        - '6385'
        - --cluster-announce-bus-port
        - '16385'
        volumes:
        - .\mount\redis-node-5:/data
        networks:
        - redis-cluster
        ports:
        - 6385:6385
        - 16385:16385
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '0.5'
                    memory: 512mb
        depends_on:
            redis-node-1:
                condition: service_started
            redis-node-2:
                condition: service_started
            redis-node-3:
                condition: service_started
            redis-node-4:
                condition: service_started
        healthcheck:
            test:
            - CMD
            - redis-cli
            - -p
            - '6385'
            - ping
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 20s
    redis-node-6:
        image: redis:8.4.0-bookworm
        container_name: redis-node-6
        command:
        - redis-server
        - --cluster-enabled
        - 'yes'
        - --requirepass
        - redis
        - --masterauth
        - redis
        - --user
        - redis
        - 'on'
        - '>redis'
        - ~*
        - +@all
        - --cluster-node-timeout
        - '15000'
        - --cluster-config-file
        - nodes.conf
        - --cluster-node-timeout
        - '5000'
        - --appendonly
        - 'yes'
        - --cluster-replica-validity-factor
        - '0'
        - --port
        - '6386'
        - --bind
        - 0.0.0.0
        - --protected-mode
        - 'no'
        - --cluster-announce-ip
        - 10.15.20.2
        - --cluster-announce-port
        - '6386'
        - --cluster-announce-bus-port
        - '16386'
        volumes:
        - .\mount\redis-node-6:/data
        networks:
        - redis-cluster
        ports:
        - 6386:6386
        - 16386:16386
        dns: *id001
        deploy:
            resources:
                limits:
                    cpus: '0.5'
                    memory: 512mb
        depends_on:
            redis-node-1:
                condition: service_started
            redis-node-2:
                condition: service_started
            redis-node-3:
                condition: service_started
            redis-node-4:
                condition: service_started
            redis-node-5:
                condition: service_started
        healthcheck:
            test:
            - CMD
            - redis-cli
            - -p
            - '6386'
            - ping
            interval: 10s
            timeout: 10s
            retries: 10
            start_period: 20s
networks:
    redis-cluster:
        driver: bridge

```

In [6]:
!docker compose -f redis-cluster.docker-compose.yml up -d --wait

 Network redis-cluster_redis-cluster  Creating
 Network redis-cluster_redis-cluster  Created
 Container redis-node-1  Creating
 Container redis-node-1  Created
 Container redis-node-2  Creating
 Container redis-node-2  Created
 Container redis-node-3  Creating
 Container redis-node-3  Created
 Container redis-node-4  Creating
 Container redis-node-4  Created
 Container redis-node-5  Creating
 Container redis-node-5  Created
 Container redis-node-6  Creating
 Container redis-node-6  Created
 Container redis-node-1  Starting
 Container redis-node-1  Started
 Container redis-node-2  Starting
 Container redis-node-2  Started
 Container redis-node-3  Starting
 Container redis-node-3  Started
 Container redis-node-4  Starting
 Container redis-node-4  Started
 Container redis-node-5  Starting
 Container redis-node-5  Started
 Container redis-node-6  Starting
 Container redis-node-6  Started
 Container redis-node-4  Waiting
 Container redis-node-5  Waiting
 Container redis-node-6  Waiting
 Con

In [7]:
nodes_ports = ' '.join([f"{REDIS_NODE_IPS[i]}:{REDIS_NODE_PORTS[i]}" for i in range(REDIS_TOTAL_NODES)])

!docker exec redis-node-1 redis-cli -a {REDIS_ADMIN_PASSWORD} --cluster create {nodes_ports} --cluster-replicas 1 --cluster-yes

>>> Performing hash slots allocation on 6 nodes...
Master[0] -> Slots 0 - 5460
Master[1] -> Slots 5461 - 10922
Master[2] -> Slots 10923 - 16383
Adding replica 10.15.20.2:6385 to 10.15.20.2:6381
Adding replica 10.15.20.2:6386 to 10.15.20.2:6382
Adding replica 10.15.20.2:6384 to 10.15.20.2:6383
>>> Trying to optimize slaves allocation for anti-affinity
[WARNING] Some slaves are in the same host as their master
M: ba35db5b6a05142a4314bf359b2e2e637786f004 10.15.20.2:6381
   slots:[0-5460] (5461 slots) master
M: 9a2ca32aabbd546042bcd5e5493e98207065a4e9 10.15.20.2:6382
   slots:[5461-10922] (5462 slots) master
M: dd7a179a96abf36857152722ae88b1ddc457d3d5 10.15.20.2:6383
   slots:[10923-16383] (5461 slots) master
S: 6560867dc47c57cb10dc394c960e6a7e3debe533 10.15.20.2:6384
   replicates ba35db5b6a05142a4314bf359b2e2e637786f004
S: d96b4f5ed5aaa54549ce5c921e8d6a7f609dad55 10.15.20.2:6385
   replicates 9a2ca32aabbd546042bcd5e5493e98207065a4e9
S: 504f4bd5dddf7d12f5ea00250884f902632237fb 10.15.20.2: